In [1]:
import param
import panel as pn
pn.extension('katex')

In [2]:
pipeline = pn.pipeline.Pipeline()

In [3]:
class Stage1(param.Parameterized):
    a = param.Number(default=5, bounds=(0, 10))
    b = param.Number(default=5, bounds=(0, 10))

    ready = param.Boolean(default=False, precedence=-1)
    
    @param.output(('c', param.Number), ('d', param.Number))
    def output(self):
        return self.a * self.b, self.a ** self.b

    @param.depends('a', 'b')
    def view(self):
        c, d = self.output()
        return pn.pane.LaTeX('${a} * {b} = {c}$\n${a}^{{{b}}} = {d}$'.format(
            a=self.a, b=self.b, c=c, d=d), style={'font-size': '2em'})

    def panel(self):
        return pn.Row(self.param, self.view)

stage1 = Stage1()
stage1.panel()
        


BokehModel(combine_events=True, render_bundle={'docs_json': {'4e62b31b-1f2c-4cc5-bfbe-72d76d653b69': {'defs': …

Row
    [0] Column(margin=5, name='Stage', width=300)
        [0] StaticText(value='<b>Stage</b>')
        [1] FloatSlider(end=10, name='A', value=5)
        [2] FloatSlider(end=10, name='B', value=5)
    [1] ParamMethod(method)

In [4]:
stage1.param.outputs()

{'c': (<param.Number at 0x1f8e1e34348>,
  <bound method Stage1.output of Stage1(a=5, b=5, name='Stage101765', ready=False)>,
  0),
 'd': (<param.Number at 0x1f8e1e34408>,
  <bound method Stage1.output of Stage1(a=5, b=5, name='Stage101765', ready=False)>,
  1)}

In [5]:
pipeline.add_stage('Stage 1', stage1)

In [6]:
class Stage2(param.Parameterized):
    c = param.Number(default=5, bounds=(0, None))
    exp = param.Number(default=0.1, bounds=(0, 3))

    @param.depends('c', 'exp')
    def view(self):
        return pn.pane.LaTeX('${%s}^{%s}={%.3f}$' % (self.c, self.exp, self.c**self.exp),
                             style={'font-size': '2em'})

    def panel(self):
        return pn.Row(self.param, self.view)


stage2 = Stage2(c=stage1.output()[0])
stage2.panel()

BokehModel(combine_events=True, render_bundle={'docs_json': {'757d2202-b843-43f7-8e2a-88d8c15c8bb7': {'defs': …

Row
    [0] Column(margin=5, name='Stage', width=300)
        [0] StaticText(value='<b>Stage</b>')
        [1] FloatInput(name='C', start=0, value=25)
        [2] FloatSlider(end=3, name='Exp', value=0.1)
    [1] ParamMethod(method)

In [7]:
pipeline.add_stage('Stage 2', stage2)

In [8]:
print(pipeline)

Pipeline:
    [0] Stage 1: Stage1()
    [1] Stage 2: Stage2(c=25)


In [9]:
pipeline = pn.pipeline.Pipeline(debug=True)
pipeline.add_stage('Stage 1', Stage1())
pipeline.add_stage('Stage 2', Stage2)

pipeline

BokehModel(combine_events=True, render_bundle={'docs_json': {'94c8e369-2103-4bfb-b041-faedfc29544d': {'defs': …

Pipeline:
    [0] Stage 1: Stage1()
    [1] Stage 2: Stage2()

In [11]:
pipeline = pn.pipeline.Pipeline([('Stage 1', Stage1), ('Stage 2', Stage2)])
pipeline

BokehModel(combine_events=True, render_bundle={'docs_json': {'6d45b08d-6d8c-4968-adf4-47f35f29bbfe': {'defs': …

Pipeline:
    [0] Stage 1: Stage1()
    [1] Stage 2: Stage2()

In [12]:
class Input(param.Parameterized):

    value1 = param.Integer(default=2)
    value2 = param.Integer(default=3)

    def panel(self):
        return pn.Row(self.param.value1, self.param.value2)

class Multiply(Input):

    def panel(self):
        return '%.3f * %.3f' % (self.value1, self.value2)
    
    @param.output('result')
    def output(self):
        return self.value1 * self.value2

class Add(Input):

    def panel(self):
        return '%d + %d' % (self.value1, self.value2)

    @param.output('result')
    def output(self):
        return self.value1 + self.value2

class Result(Input):

    result = param.Number(default=0)

    def panel(self):
        return self.result

dog = pn.pipeline.Pipeline()

dog.add_stage('Input', Input)
dog.add_stage('Multiply', Multiply)
dog.add_stage('Add', Add)
dog.add_stage('Result', Result)



In [13]:
dog.define_graph({'Input': ('Multiply', 'Add'), 'Multiply': 'Result', 'Add': 'Result'})

In [14]:
dog

BokehModel(combine_events=True, render_bundle={'docs_json': {'04441b19-df63-436b-9e77-698558325b61': {'defs': …

Pipeline:
    [0] Input: Input()
    [1] Multiply: Multiply()
    [2] Add: Add()
    [3] Result: Result()

In [16]:
pn.Column(
    pn.Row(dog.title, pn.layout.HSpacer(), dog.buttons),
    dog.network,
    dog.stage
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'9bd2322e-72e1-48c8-9f47-9338542b7180': {'defs': …

Column
    [0] Row
        [0] Markdown(str, margin=(0, 0, 0, 5))
        [1] HSpacer()
        [2] Row
            [0] Column
                [0] Select(disabled=True, width=125)
                [1] Button(clicks=4, disabled=True, name='Previous', width=125)
            [1] Column
                [0] Select(options=['Multiply', 'Add'], value='Multiply', width=125)
                [1] Button(clicks=3, name='Next', width=125)
    [1] HoloViews(Overlay, backend='bokeh')
    [2] Row
        [0] Row
            [0] IntInput(name='Value1', value=5)
            [1] IntInput(name='Value2', value=3)